<a href="https://colab.research.google.com/github/maminu0/maminu0/blob/main/teacher_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: read data from my google drive and list items and paths of item in the drive

from google.colab import drive
drive.mount('/content/drive')

import os
# root_dir = "/content/drive/My Drive/"
# for dir_name, _, file_names in os.walk(root_dir):
#   for file_name in file_names:
#     print(os.path.join(dir_name, file_name))


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.compat.v1 import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape, GlobalMaxPool2D, GRU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
#import numpy as np
import warnings, time
warnings.simplefilter('ignore')


In [ ]:

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/mmd_dataset.csv')
data.head()

# GLOBAL VARIABLES
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=['-',':','-.','--','-',':','-.','--','-',':','-.','--','-']


In [ ]:
dataX = data.loc[:, ~data.columns.str.contains('^Unnamed')]
ytrain = data["Class"]
styles=['-=-',':','-.','-.-','-',':','-.','--','-',':','-.','--','-']
ytrain.head()

,Class
0,4
1,9
2,6
3,4
4,5


In [ ]:
features = ['0','1','2','3','4','5','6','7','8','9','0a','0b','0c','0d','0e','0f','10','11','12','13','14','15','16','17','18','19',
            '1a','1b','1c','1d','1e','1f','20','21','22','23','24','25','26','27','28','29','2a','2b','2c','2d','2e','2f','30','31',
            '32','33','34','35','36','37','38','39','3a','3b','3c','3d','3e','3f','40','41','42','43','44','45','46','47','48','49',
            '4a','4b','4c','4d','4e','4f','50','51','52','53','54','55','56','57','58','59','5a','5b','5c','5d','5e','5f','60','61',
            '62','63','64','65','66','67','68','69','6a','6b','6c','6d','6e','6f','70','71','72','73','74','75','76','77','78','79',
            '7a','7b','7c','7d','7e','7f','80','81','82','83','84','85','86','87','88','89','8a','8b','8c','8d','8e','8f','90','91',
            '92','93','94','95','96','97','98','99','9a','9b','9c','9d','9e','9f','a0','a1','a2','a3','a4','a5','a6','a7','a8','a9',
            'aa','ab','ac','ad','ae','af','b0','b1','b2','b3','b4','b5','b6','b7','b8','b9','ba','bb','bc','bd','be','bf','c0','c1',
            'c2','c3','c4','c5','c6','c7','c8','c9','ca','cb','cc','cd','ce','cf','d0','d1','d2','d3','d4','d5','d6','d7','d8','d9',
            'da','db','dc','dd','de','df','e0','e1','e2','e3','e4','e5','e6','e7','e8','e9','ea','eb','ec','ed','ee','ef','f0','f1',
            'f2','f3','f4','f5','f6','f7','f8','f9',"fa","fb","fc","fd","fe","ff","??", 'HEADER:', '.text:', '.Pav:', '.idata:',
                '.data:', '.bss:', '.rdata:', '.edata:', '.rsrc:', '.tls:', '.reloc:', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor',
            'retn', 'nop', 'sub', 'inc', 'dec', 'add', 'imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb', 'jz',
            'lea', 'movzx', '.dll', 'std::', ':dword', 'edx', 'esi', 'eax', 'ebx', 'ecx', 'edi', 'ebp', 'esp', 'eip']
dataX=data[features]
dataX.head()

,0,1,2,3,4,5,6,7,8,9,...,:dword,edx,esi,eax,ebx,ecx,edi,ebp,esp,eip
0,3552,439,437,473,538,426,419,420,402,392,...,747,627,496,476,167,285,0,1,0,10
1,94955,1057,591,658,554,723,455,470,654,370,...,3581,579,1872,815,538,883,0,30,12,498
2,9940,1167,666,700,560,582,644,639,521,511,...,366,133,215,134,109,318,0,23,0,51
3,4130,366,418,494,501,417,416,405,402,359,...,825,600,675,643,145,296,0,3,0,20
4,9838,1807,1507,1276,1822,1346,1240,1033,1494,1216,...,675,273,243,220,379,61,0,9,0,73


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(ytrain)

y_train = to_categorical(y_train)

# GLOBAL VARIABLES
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, GlobalAvgPool2D, Reshape, Bidirectional, LSTM, Dense
from sklearn.metrics import accuracy_score


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Define callbacks list
callbacks = [annealer]#callbacks = [early_stopping, annealer]#
X_train, X_val, y_train_, y_val = train_test_split(dataX, y_train, test_size = 0.2, stratify=y_train, random_state=42)


In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
def plot_(hist, name="teacher_pattern_plot"):
    fig , ax = plt.subplots(1,2)
    train_acc = hist.history['accuracy']
    train_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    fig.set_size_inches(16,9)

    ax[0].plot( train_acc , 'go-' , label = 'Training Accuracy')
    ax[0].plot( val_acc , 'ro-' , label = 'Testing Accuracy')
    ax[0].set_title('Training & Validation Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")

    ax[1].plot( train_loss , 'g-o' , label = 'Training Loss')
    ax[1].plot( val_loss , 'r-o' , label = 'Testing Loss')
    ax[1].set_title('Testing Accuracy & Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    plt.savefig(f"{'_'.join(name)}_{str(time.time())}_{name}_.png")
    plt.show()

def pre_cl(model,X_val2,Y_val2, name="teacher"):
    predictions = model.predict(X_val2)
    Xpred = np.argmax(predictions, axis=1)
    ytrue = np.argmax(Y_val2, axis=1)
    print(classification_report(ytrue, Xpred))
    cm = confusion_matrix(ytrue,Xpred)
    sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')
    plt.figure(figsize = (15,15))
    plt.savefig(f"{'_'.join(name)}_{str(time.time())}_{name}_.png")
    plt.show()

def plot_roc(model,X_test,Y_test, name="final"):

    # Assuming you have predictions from your model
    y_pred = model.predict(X_test)  # Replace X_test with your test data

    # Assuming your labels are one-hot encoded
    y_true = np.argmax(Y_test, axis=1)  # Get the class labels from one-hot encoded Y_test

    # Calculate the ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_classes = Y_test.shape[1]
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot the ROC curve for each class
    plt.figure()
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                  .format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    #plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig(f"{'_'.join(name)}_{str(time.time())}_{name}_.png")
    plt.show()


In [ ]:
import tensorflow as tf
def inception_module(input_tensor, n_filters=128):
    # Bottleneck layer
    bottleneck = layers.Conv1D(n_filters, kernel_size=1, kernel_regularizer=tf.keras.regularizers.l2(0.001), padding='same')(input_tensor)
    bottleneck = layers.BatchNormalization()(bottleneck)
    bottleneck = layers.Activation('relu')(bottleneck)
    bottleneck = layers.Dropout(0.1)(bottleneck)

    # Inception layers
    conv1 = layers.Conv1D(n_filters, kernel_size=1, padding='same')(bottleneck)
    conv3 = layers.Conv1D(n_filters, kernel_size=3, padding='same')(bottleneck)
    conv5 = layers.Conv1D(n_filters, kernel_size=5, padding='same')(bottleneck)
    maxpool = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(bottleneck)
    maxpool = layers.Conv1D(n_filters, kernel_size=1, padding='same')(maxpool)

    # Concatenate
    output = layers.concatenate([conv1, conv3, conv5, maxpool], axis=-1)
    return output

def InceptionTime(input_shape, n_classes=2, n_filters=128, n_modules=6):
    inputs = layers.Input(shape=input_shape)

    x = inputs
    for _ in range(n_modules):
        x = inception_module(x, n_filters=n_filters)
        x = layers.Dropout(0.1)(x)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (305, 1)  # Example input shape: 100 time steps, 1 feature
n_classes = 9  # Number of classes for classification
Inceptionmodel = InceptionTime(input_shape=input_shape, n_classes=n_classes, n_filters=128, n_modules=6)
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
Inceptionmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
Inceptionmodel.summary()
Inceptionmodel_history = Inceptionmodel.fit(X_train, y_train_, epochs=200, batch_size=32, verbose=2, validation_data=(X_val, y_val), callbacks=[annealer])


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 305, 1)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 305, 128)       │            256 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 305, 128)       │            512 │ conv1d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 305, 128)       │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 305, 128)       │              0 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 305, 128)       │              0 │ dropout[0][0]          │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 305, 128)       │         16,512 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 305, 128)       │         49,280 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 305, 128)       │         82,048 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 305, 128)       │         16,512 │ max_pooling1d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 305, 512)       │              0 │ conv1d_1[0][0],        │
│                           │                        │                │ conv1d_2[0][0],        │
│                           │                        │                │ conv1d_3[0][0],        │
│                           │                        │                │ conv1d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 305, 512)       │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 305, 128)       │         65,664 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 305, 128)       │            512 │ conv1d_5[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 305, 128)       │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 305, 128)       │              0 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1      

 Total params: 1,322,377 (5.04 MB)

 Trainable params: 1,320,841 (5.04 MB)

 Non-trainable params: 1,536 (6.00 KB)

Epoch 1/200
272/272 - 52s - 193ms/step - accuracy: 0.6920 - loss: 1.5040 - val_accuracy: 0.2861 - val_loss: 5.8164 - learning_rate: 0.0010
Epoch 2/200
272/272 - 10s - 38ms/step - accuracy: 0.7557 - loss: 0.9919 - val_accuracy: 0.3634 - val_loss: 5.9594 - learning_rate: 9.5000e-04
Epoch 3/200
272/272 - 20s - 74ms/step - accuracy: 0.8009 - loss: 0.7748 - val_accuracy: 0.4259 - val_loss: 3.2169 - learning_rate: 9.0250e-04
Epoch 4/200
272/272 - 10s - 38ms/step - accuracy: 0.8164 - loss: 0.7142 - val_accuracy: 0.2755 - val_loss: 15.2195 - learning_rate: 8.5737e-04
Epoch 5/200
272/272 - 10s - 37ms/step - accuracy: 0.8003 - loss: 0.7471 - val_accuracy: 0.6265 - val_loss: 1.2698 - learning_rate: 8.1451e-04
Epoch 6/200
272/272 - 10s - 38ms/step - accuracy: 0.8305 - loss: 0.6361 - val_accuracy: 0.4503 - val_loss: 2.0163 - learning_rate: 7.7378e-04
Epoch 7/200
272/272 - 21s - 76ms/step - accuracy: 0.8481 - loss: 0.5675 - val_accuracy: 0.5414 - val_loss: 1.6905 - learning_rate: 7.3509e-04
Epoch 8/

In [ ]:
def CNN_GRU_Dense_InceptionTime(input_shape, n_classes=2, n_filters=128, n_modules=6):
    inputs = layers.Input(shape=input_shape)

    # CNN layers
    x = layers.Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001),  padding='same')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001),  padding='same')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # InceptionTime modules
    for _ in range(n_modules):
        x = inception_module(x, n_filters=n_filters)
        x = layers.Dropout(0.2)(x)

    # GRU layer
    x = layers.GRU(128, return_sequences=True)(x)
    x = layers.GlobalMaxPooling1D()(x)

    # Dense layers
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (305, 1)  # Example input shape: 100 time steps, 1 feature
n_classes = 9  # Number of classes for classification
CNN_GRU_Dense_model = CNN_GRU_Dense_InceptionTime(input_shape=input_shape, n_classes=n_classes, n_filters=128, n_modules=6)
CNN_GRU_Dense_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
CNN_GRU_Dense_model.summary()
teacher_hybrid_model = tf.keras.models.clone_model(CNN_GRU_Dense_model)
teacher_hybrid_model_history = CNN_GRU_Dense_model.fit(X_train, y_train_, epochs=200, batch_size=32, verbose=2, validation_data=(X_val, y_val), callbacks=[annealer])



Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 305, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_62 (Conv1D)        │ (None, 305, 64)        │            256 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_14          │ (None, 152, 64)        │              0 │ conv1d_62[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_63 (Conv1D)        │ (None, 152, 128)       │         24,704 │ max_pooling1d_14[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_15          │ (None, 76, 128)        │              0 │ conv1d_63[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_64 (Conv1D)        │ (None, 76, 128)        │         16,512 │ max_pooling1d_15[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_12    │ (None, 76, 128)        │            512 │ conv1d_64[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_12             │ (None, 76, 128)        │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 76, 128)        │              0 │ activation_12[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_16          │ (None, 76, 128)        │              0 │ dropout_25[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_65 (Conv1D)        │ (None, 76, 128)        │         16,512 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_66 (Conv1D)        │ (None, 76, 128)        │         49,280 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_67 (Conv1D)        │ (None, 76, 128)        │         82,048 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_68 (Conv1D)        │ (None, 76, 128)        │         16,512 │ max_pooling1d_16[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_12            │ (None, 76, 512)        │              0 │ conv1d_65[0][0],       │
│ (Concatenate)             │                        │                │ conv1d_66[0][0],       │
│                           │                        │                │ conv1d_67[0][0],       │
│                           │                        │                │ conv1d_68[0][0]        │
├──────────────────────

 Total params: 1,623,177 (6.19 MB)

 Trainable params: 1,621,641 (6.19 MB)

 Non-trainable params: 1,536 (6.00 KB)

Epoch 1/200
272/272 - 35s - 129ms/step - accuracy: 0.6092 - loss: 2.0010 - val_accuracy: 0.6389 - val_loss: 1.6537 - learning_rate: 0.0010
Epoch 2/200
272/272 - 14s - 50ms/step - accuracy: 0.7792 - loss: 1.1320 - val_accuracy: 0.7507 - val_loss: 1.1054 - learning_rate: 9.5000e-04
Epoch 3/200
272/272 - 10s - 36ms/step - accuracy: 0.8496 - loss: 0.7744 - val_accuracy: 0.8165 - val_loss: 0.7766 - learning_rate: 9.0250e-04
Epoch 4/200
272/272 - 11s - 39ms/step - accuracy: 0.8751 - loss: 0.6133 - val_accuracy: 0.7098 - val_loss: 1.1219 - learning_rate: 8.5737e-04
Epoch 5/200
272/272 - 9s - 32ms/step - accuracy: 0.8930 - loss: 0.5398 - val_accuracy: 0.8855 - val_loss: 0.5334 - learning_rate: 8.1451e-04
Epoch 6/200
272/272 - 11s - 41ms/step - accuracy: 0.9017 - loss: 0.4735 - val_accuracy: 0.8109 - val_loss: 0.6934 - learning_rate: 7.7378e-04
Epoch 7/200
272/272 - 10s - 36ms/step - accuracy: 0.9068 - loss: 0.4474 - val_accuracy: 0.9016 - val_loss: 0.3888 - learning_rate: 7.3509e-04
Epoch 8/20

In [ ]:
def GRU_Inception(input_shape, n_classes=2, n_filters=128, n_modules=6):
    inputs = layers.Input(shape=input_shape)

    x = inputs
    for _ in range(n_modules):
        # Inception module with GRU
        bottleneck = layers.Conv1D(n_filters, kernel_size=1, padding='same')(x)
        bottleneck = layers.BatchNormalization()(bottleneck)
        bottleneck = layers.Activation('relu')(bottleneck)


        # Apply GRU to each branch
        conv1 = GRU(n_filters//6, return_sequences=True)(bottleneck)
        conv3 = GRU(n_filters//4, return_sequences=True)(bottleneck)
        conv5 = GRU(n_filters//2, return_sequences=True)(bottleneck)
        maxpool = GRU(n_filters, return_sequences=True)(bottleneck)

        # Concatenate
        x = layers.concatenate([conv1, conv3, conv5, maxpool], axis=-1)
        x = layers.Dropout(0.2)(x)

    x = layers.GlobalMaxPooling1D()(x)
    x = Reshape((1, 245))(x)

    x = layers.Bidirectional(GRU(n_filters, return_sequences=False))(x)

    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (305, 1)  # Example input shape: 100 time steps, 1 feature
n_classes = 9  # Number of classes for classification
gru_model = GRU_Inception(input_shape=input_shape, n_classes=n_classes, n_filters=128, n_modules=6)
gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
gru_model.summary()
gru_clone = tf.keras.models.clone_model(gru_model)
gru_clone_history = gru_model.fit(X_train, y_train_, epochs=200, batch_size=32, verbose=2, validation_data=(X_val, y_val), callbacks=[annealer])


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 305, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_94 (Conv1D)        │ (None, 305, 128)       │            256 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_18    │ (None, 305, 128)       │            512 │ conv1d_94[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_18             │ (None, 305, 128)       │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_2 (GRU)               │ (None, 305, 21)        │          9,513 │ activation_18[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_3 (GRU)               │ (None, 305, 32)        │         15,552 │ activation_18[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_4 (GRU)               │ (None, 305, 64)        │         37,248 │ activation_18[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_5 (GRU)               │ (None, 305, 128)       │         99,072 │ activation_18[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_18            │ (None, 305, 245)       │              0 │ gru_2[0][0],           │
│ (Concatenate)             │                        │                │ gru_3[0][0],           │
│                           │                        │                │ gru_4[0][0],           │
│                           │                        │                │ gru_5[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_38 (Dropout)      │ (None, 305, 245)       │              0 │ concatenate_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_95 (Conv1D)        │ (None, 305, 128)       │         31,488 │ dropout_38[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_19    │ (None, 305, 128)       │            512 │ conv1d_95[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_19             │ (None, 305, 128)       │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_6 (GRU)               │ (None, 305, 21)        │          9,513 │ activation_19[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_7 (GRU)               │ (None, 305, 32)        │         15,552 │ activation_19[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_8 (GRU)          

 Total params: 1,419,391 (5.41 MB)

 Trainable params: 1,417,855 (5.41 MB)

 Non-trainable params: 1,536 (6.00 KB)

Epoch 1/200
272/272 - 90s - 329ms/step - accuracy: 0.7904 - loss: 0.6397 - val_accuracy: 0.7443 - val_loss: 0.8394 - learning_rate: 0.0010
Epoch 2/200
272/272 - 71s - 262ms/step - accuracy: 0.9004 - loss: 0.3202 - val_accuracy: 0.7682 - val_loss: 0.6433 - learning_rate: 9.5000e-04
Epoch 3/200
272/272 - 78s - 287ms/step - accuracy: 0.9388 - loss: 0.2069 - val_accuracy: 0.9545 - val_loss: 0.1734 - learning_rate: 9.0250e-04
Epoch 4/200
272/272 - 82s - 301ms/step - accuracy: 0.9509 - loss: 0.1744 - val_accuracy: 0.8602 - val_loss: 0.4649 - learning_rate: 8.5737e-04
Epoch 5/200
272/272 - 81s - 298ms/step - accuracy: 0.9605 - loss: 0.1353 - val_accuracy: 0.9434 - val_loss: 0.1998 - learning_rate: 8.1451e-04
Epoch 6/200
272/272 - 82s - 301ms/step - accuracy: 0.9670 - loss: 0.1137 - val_accuracy: 0.8316 - val_loss: 0.4123 - learning_rate: 7.7378e-04
Epoch 7/200
272/272 - 83s - 305ms/step - accuracy: 0.9683 - loss: 0.1097 - val_accuracy: 0.9508 - val_loss: 0.2031 - learning_rate: 7.3509e-04
Epo

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Average, Maximum

def ensemble_model(input_shape, num_classes=9, n_filters=32, n_modules=2):

    # Create a functional API model for the ensemble
    inputs = Input(shape=input_shape)
    outputs = Maximum()([gru_model(inputs), Inceptionmodel(inputs), CNN_GRU_Dense_model(inputs)])

    ensemble_model = Model(inputs=inputs, outputs=outputs)
    ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble_model.summary()

    return ensemble_model

mnet = ensemble_model(input_shape, num_classes=9, n_filters=32, n_modules=2)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Define callbacks list
callbacks = [early_stopping, annealer]#
X_train, X_val, y_train_, y_val = train_test_split(dataX.values, y_train, test_size = 0.2, stratify=y_train, random_state=42)


In [ ]:
from sklearn.model_selection import KFold
def train_with_cross_val(model, X_train, y_train, fold=5):
    kf = KFold(n_splits=fold, shuffle=True, random_state=42)
    metrics = []

    for train_index, val_index in kf.split(X_train):
        x_train_fold, x_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        #model.fit(x_train_fold,y_train_fold)
        #callbacks = [early_stopping, annealer]

        model.fit(x_train_fold,y_train_fold, epochs=100, batch_size=128, verbose=2, validation_data=(x_val_fold, y_val_fold))
        y_pre_ = model.predict(x_val_fold)
        accuracy = np.mean(y_pre_ == y_val_fold)
        metrics.append(accuracy)
        print(f'Accuracy={accuracy}')

    return metrics


metr = train_with_cross_val(mnet, X_train, y_train_, fold=5)

In [ ]:
# Define the checkpoint path
checkpoint_path = "model_checkpoints/mtenet_max.weights.h5" # Changed the file extension to .weights.h5
#checkpoint_path = "/content/drive/My Drive/Colab Notebooks/model_checkpoints/cp.ckpt"

# Create a ModelCheckpoint callback
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 verbose=1)

# Train the model with the checkpoint callback
mtenet_history = mnet.fit(X_train, y_train_, epochs=200, batch_size=32, verbose=2, validation_data=(X_val, y_val), callbacks=[annealer, cp_callback])
# Plot the training history
plot_(mtenet_history)

# Evaluate the model on the validation set
pre_cl(mnet, X_val, y_val)
plot_roc(mnet,X_val,y_val)

In [ ]:
# Plot the training history
plot_(m_history)

# Evaluate the model on the validation set
pre_cl(model, X_val, y_val)
plot_roc(model,X_val,y_val)

In [ ]:
def inception_module(input_tensor, n_filters=128):
    # Bottleneck layer
    bottleneck = layers.Conv1D(n_filters, kernel_size=1, padding='same')(input_tensor)
    bottleneck = layers.BatchNormalization()(bottleneck)
    bottleneck = layers.Activation('relu')(bottleneck)

    # Inception layers
    conv1 = layers.Conv1D(n_filters, kernel_size=1, padding='same')(bottleneck)
    conv3 = layers.Conv1D(n_filters, kernel_size=3, padding='same')(bottleneck)
    conv5 = layers.Conv1D(n_filters, kernel_size=5, padding='same')(bottleneck)
    maxpool = layers.MaxPooling1D(pool_size=3, strides=1, padding='same')(bottleneck)
    maxpool = layers.Conv1D(n_filters, kernel_size=1, padding='same')(maxpool)

    # Add residual connection
    residual = layers.Conv1D(n_filters * 4, kernel_size=1, padding='same')(input_tensor)
    residual = layers.BatchNormalization()(residual)

    # Concatenate
    output = layers.concatenate([conv1, conv3, conv5, maxpool], axis=-1)
    output = layers.Add()([output, residual])
    output = layers.Activation('relu')(output)
    return output

def InceptionTime(input_shape, n_classes=2, n_filters=128, n_modules=6):
    inputs = layers.Input(shape=input_shape)

    x = inputs
    for _ in range(n_modules):
        x = inception_module(x, n_filters=n_filters)
        x = layers.Dropout(0.2)(x)  # Add dropout for regularization

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(256, activation='relu')(x)  # Add a dense layer
    x = layers.Dropout(0.5)(x)  # Add dropout for regularization
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Example usage:
input_shape = (305, 1)  # Example input shape: 100 time steps, 1 feature
n_classes = 9  # Number of classes for classification
base_model = InceptionTime(input_shape=input_shape, n_classes=n_classes, n_filters=128, n_modules=8)
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
base_model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/base_model.weights.h5"

# Create a ModelCheckpoint callback
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 verbose=1)

# Train the model with the checkpoint callback
base_history = base_model.fit(X_train, y_train_, epochs=100, batch_size=128, verbose=2,
                      validation_data=(X_val, y_val), callbacks=[annealer, cp_callback])

#base_history=base_model.fit(X_train, y_train_, epochs=100, batch_size=128, verbose=2, validation_data=(X_val, y_val), callbacks=callbacks)

In [ ]:
# Plot the training history
plot_(base_history)

# Evaluate the model on the validation set
pre_cl(base_model, X_val, y_val)
plot_roc(base_model,X_val,y_val)

In [ ]:
# prompt: save train model with check points to be used later

from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/model_cp.weights.h5"

# Create a ModelCheckpoint callback
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 verbose=1)

# Train the model with the checkpoint callback
m_history = model.fit(X_train, y_train_, epochs=100, batch_size=128, verbose=2,
                      validation_data=(X_val, y_val), callbacks=[annealer, cp_callback])


In [ ]:
def CNN_GRU_Dense_InceptionTime(input_shape, n_classes=2, n_filters=128, n_modules=6):
    inputs = layers.Input(shape=input_shape)

    # CNN layers
    x = layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # InceptionTime modules
    for _ in range(n_modules):
        x = inception_module(x, n_filters=n_filters)
        x = layers.Dropout(0.2)(x)

    # GRU layer
    x = layers.GRU(128, return_sequences=True)(x)
    x = layers.GlobalMaxPooling1D()(x)

    # Dense layers
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (305, 1)  # Example input shape: 100 time steps, 1 feature
n_classes = 9  # Number of classes for classification
CNN_GRU_Dense_model = CNN_GRU_Dense_InceptionTime(input_shape=input_shape, n_classes=n_classes, n_filters=128, n_modules=6)
CNN_GRU_Dense_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
CNN_GRU_Dense_model.summary()

In [ ]:
# Define the checkpoint path
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/model_checkpoints/cp_incep_cgru.weights.h5"

# Create a ModelCheckpoint callback
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 verbose=1)

# Train the model with the checkpoint callback

CNN_GRU_Dense_model_history=CNN_GRU_Dense_model.fit(X_train, y_train_, epochs=100, batch_size=128, verbose=2, validation_data=(X_val, y_val), callbacks=[annealer, cp_callback])

In [ ]:
# Plot the training history
plot_(CNN_GRU_Dense_model_history)

# Evaluate the model on the validation set
pre_cl(CNN_GRU_Dense_model, X_val, y_val)
plot_roc(CNN_GRU_Dense_model,X_val,y_val)

In [ ]:
def GRU_InceptionTime(input_shape, n_classes=2, n_filters=128, n_modules=6):
    inputs = layers.Input(shape=input_shape)

    x = inputs
    for _ in range(n_modules):
        # Inception module with GRU
        bottleneck = layers.Conv1D(n_filters, kernel_size=1, padding='same')(x)
        bottleneck = layers.BatchNormalization()(bottleneck)
        bottleneck = layers.Activation('relu')(bottleneck)

        conv1 = layers.Conv1D(n_filters, kernel_size=1, padding='same')(bottleneck)
        conv3 = layers.Conv1D(n_filters, kernel_size=3, padding='same')(bottleneck)
        conv5 = layers.Conv1D(n_filters, kernel_size=5, padding='same')(bottleneck)
        maxpool = layers.MaxPooling1D(pool_size=3, strides=1, padding='same')(bottleneck)
        maxpool = layers.Conv1D(n_filters, kernel_size=1, padding='same')(maxpool)

        # Apply GRU to each branch
        conv1 = layers.Bidirectional(GRU(n_filters // 2, return_sequences=True))(conv1)
        conv3 = layers.Bidirectional(GRU(n_filters // 2, return_sequences=True))(conv3)
        conv5 = layers.Bidirectional(GRU(n_filters // 2, return_sequences=True))(conv5)
        maxpool = layers.Bidirectional(GRU(n_filters // 2, return_sequences=True))(maxpool)

        # Concatenate
        x = layers.concatenate([conv1, conv3, conv5, maxpool], axis=-1)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (305, 1)  # Example input shape: 100 time steps, 1 feature
n_classes = 9  # Number of classes for classification
gru_model = GRU_InceptionTime(input_shape=input_shape, n_classes=n_classes, n_filters=128, n_modules=6)
gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
gru_model.summary()

In [ ]:
# Define the checkpoint path
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/gru_cp.weight.h5"

# Create a ModelCheckpoint callback
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 verbose=1)

# Train the model with the checkpoint callback
gru_history = gru_model.fit(X_train, y_train_, epochs=100, batch_size=128, verbose=2,
                      validation_data=(X_val, y_val), callbacks=[annealer, cp_callback])
#gru_history=gru_model.fit(X_train, y_train_, epochs=100, batch_size=128, verbose=2, validation_data=(X_val, y_val), callbacks=callbacks)

In [ ]:
# Plot the training history
plot_(gru_history)

# Evaluate the model on the validation set
pre_cl(gru_model, X_val, y_val)
plot_roc(gru_model,X_val,y_val)

In [ ]:
def ensemble_model(input_shape, num_classes=9, n_filters=32, n_modules=6):
    # Create individual models
    inception_gru_model = build_inceptionGRU_model(input_shape, num_classes, n_filters, n_modules)
    cnn_gru_dense_inception_time_model = CNN_GRU_Dense_InceptionTime(input_shape, num_classes, n_filters, n_modules)
    inception_time_model = base_model(input_shape, num_classes, n_filters, n_modules)

    # Compile each model with a common optimizer and loss function
    optimizer = Adam(learning_rate=0.001)  # Adjust learning rate as needed
    loss_function = CategoricalCrossentropy()  # Replace with appropriate loss if necessary

    inception_gru_model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
    cnn_gru_dense_inception_time_model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
    inception_time_model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

    # Create a functional API model for the ensemble
    inputs = Input(shape=input_shape)
    outputs = Average()([inception_gru_model(inputs), cnn_gru_dense_inception_time_model(inputs), inception_time_model(inputs)])

    ensemble_model = Model(inputs=inputs, outputs=outputs)

    return ensemble_model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Average, Lambda
from tensorflow.keras.models import Model

# Define the ensemble model as the teacher (already defined in previous response)
ensemble_model = ensemble_model(input_shape=(305, 1), num_classes=num_class)  # Adjust input shape and num_classes

# Define the student model (your CNN_model)
student_model = CNN_model(window_size=305)  # Assuming window_size is 305

# Compile the student model (optional, for separate training if needed)
student_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# Function to compute the softened logits (teacher predictions)
def temperature_softmax(predictions, temperature=4.0):
    """
    Apply temperature scaling to predictions.
    """
    return tf.nn.softmax(predictions / temperature)

# Define the knowledge distillation loss
def knowledge_distillation_loss(y_true, y_pred, teacher_predictions):
    """
    Combine the cross-entropy loss with the distillation loss.
    """
    alpha = 0.5  # Weight for the distillation loss (adjust as needed)
    student_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, y_pred)
    teacher_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, teacher_predictions)
    soft_teacher_predictions = temperature_softmax(teacher_predictions)
    distillation_loss = alpha * tf.keras.losses.kullback_leibler_divergence(soft_teacher_predictions, y_pred)
    return student_loss + distillation_loss

# Create a functional model for training the student with distillation
inputs = Input(shape=(305, 1))
student_predictions = student_model(inputs)
teacher_predictions = ensemble_model(inputs)  # Pass input to the ensemble model
loss = Lambda(knowledge_distillation_loss, arguments={'teacher_predictions': teacher_predictions})(student_predictions)
distillation_model = Model(inputs=inputs, outputs=loss)

# Compile the distillation model (using the custom loss function)
distillation_model.compile(optimizer="adam", loss=lambda y_true, y_pred: y_pred)  # Ignoring true labels

# Train the student model using the distillation loss
distillation_model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, X_val))

# After training, the student_model should have improved performance due to knowledge transfer

In [ ]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Assuming you have three models: model1, model2, model3

# Create a VotingClassifier with 'soft' voting (weighted average of probabilities)
ensemble_model = VotingClassifier(estimators=[
    ('model', model),
    ('model_base', model_base),
    ('gru_model', gru_model),
    ('CNN_GRU_Dense_model', CNN_GRU_Dense_model)
], voting='soft')

# Fit the ensemble model on the training data
ensemble_history=ensemble_model.fit(X_train, np.argmax(y_train, axis=1))  # Use argmax to get the class labels

# Make predictions on the validation set
ensemble_predictions = ensemble_model.predict(X_val)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(np.argmax(y_val, axis=1), ensemble_predictions)
print("Ensemble Model Accuracy:", ensemble_accuracy)

In [ ]:
# Plot the training history
plot_(ensemble_history)

# Evaluate the model on the validation set
pre_cl(ensemble_model, X_val, y_val)
plot_roc(ensemble_model,X_val,y_val)

In [ ]:
#how do i better the InceptionTime model created above? What can i do to optimize it?

In [ ]:
# prompt: auto download saved figures, assuming training is done on kaggle platforms

!find . -name "*.png" -exec bash -c 'cat {} | xargs echo -n "{}" > /kaggle/working/files.txt' \;
!cat files.txt | while IFS= read -r line; do kaggle competitions submissions -c [YOUR_COMPETITION_NAME] -f "$line" -m "My submission"; done


In [ ]:
nets = 5
model = [0] *nets

j=0
#model[j] = Sequential()
input_shape = Input(shape=(305, 1))
#input_shape=(305,1)
model[j]=layers.Conv1D(32,kernel_size=5,activation='relu')(input_shape)
model[j]=layers.MaxPool1D()(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Conv1D(64,kernel_size=5,activation='relu')(model[j])
model[j]=layers.GlobalMaxPool1D()(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Reshape((1, 64))(model[j])
model[j]=layers.GRU(128, activation='relu')(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Dense(10, activation='softmax')(model[j])
model[j] = Model(inputs=input_shape, outputs=model[j])
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model[j].summary()

j=1
#model[j] = Sequential()
model[j]=layers.Conv1D(32,kernel_size=3,activation='relu')(input_shape)
model[j]=layers.Conv1D(32,kernel_size=3,activation='relu')(model[j])
model[j]=layers.MaxPool1D()(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Conv1D(64,kernel_size=3,activation='relu')(model[j])
model[j]=layers.Conv1D(64,kernel_size=3,activation='relu')(model[j])
model[j]=layers.GlobalMaxPool1D()(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Reshape((1, 64))(model[j])
model[j]=layers.GRU(128, activation='relu')(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Dense(10, activation='softmax')(model[j])
model[j] = Model(inputs=input_shape, outputs=model[j])
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model[j].summary()

j=2
model[j]=layers.Conv1D(32,kernel_size=5,activation='relu')(input_shape)
model[j]=layers.Conv1D(32,kernel_size=5,strides=2,padding='same',activation='relu')(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Conv1D(64,kernel_size=5,activation='relu')(model[j])
model[j]=layers.Conv1D(64,kernel_size=5,strides=2,padding='same',activation='relu')(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.GlobalMaxPool1D()(model[j])
model[j]=layers.Reshape((1, 64))(model[j])
model[j]=layers.GRU(128, activation='relu')(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Dense(10, activation='softmax')(model[j])
model[j] = Model(inputs=input_shape, outputs=model[j])
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model[j].summary()

j=3
model[j]=layers.Conv1D(32,kernel_size=3,activation='relu')(input_shape)
model[j]=layers.BatchNormalization()(model[j])
model[j]=layers.Conv1D(32,kernel_size=3,activation='relu')(model[j])
model[j]=layers.BatchNormalization()(model[j])
model[j]=layers.Conv1D(32,kernel_size=5,strides=2,padding='same',activation='relu')(model[j])
model[j]=layers.BatchNormalization()(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Conv1D(64,kernel_size=3,activation='relu')(model[j])
model[j]=layers.BatchNormalization()(model[j])
model[j]=layers.Conv1D(64,kernel_size=3,activation='relu')(model[j])
model[j]=layers.BatchNormalization()(model[j])
model[j]=layers.Conv1D(64,kernel_size=5,strides=2,padding='same',activation='relu')(model[j])
model[j]=layers.BatchNormalization()(model[j])
model[j]=layers.GlobalMaxPool1D()(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Reshape((1, 64))(model[j])
model[j]=layers.GRU(128, activation='relu')(model[j])
model[j]=layers.Dropout(0.4)(model[j])
model[j]=layers.Dense(10, activation='softmax')(model[j])
model[j] = Model(inputs=input_shape, outputs=model[j])
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model[j].summary()


In [ ]:
model = InceptionTime(input_shape=input_shape, n_classes=n_classes, n_filters=128, n_modules=6)
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
def check_com(model):

    _params = model.count_params()
    print(model.summary())

    print(f"Parameters: {_params}")

# Code cell <undefined>
# # %% [code]
# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 3
model = [0] *nets

for j in range(3):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,padding='same',activation='relu',
            input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    if j>0:
        model[j].add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D())
    if j>1:
        model[j].add(Conv2D(96,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D(padding='same'))
    model[j].add(Flatten())
    #if j>0:

    #model[j].add(Reshape((1, 64)))
    #model[j].add(GRU(128, activation='relu'))

    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])


In [ ]:
# CREATE VALIDATION SET
names = ["(C-P)x1","(C-P)x2","(C-P)x3"]
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)
# # TRAIN NETWORKS
# history = [0] * nets
#
# epochs = 20
# for j in range(nets):
#     history[j] = model[j].fit(X_train2,Y_train2, batch_size=128, epochs = epochs,
#         validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
#     print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         names[j],epochs,max(history[j].history['accuracy']),max(history[j].history['val_accuracy']) ))


In [ ]:
# # PLOT ACCURACIES
# plt.figure(figsize=(15,5))
# for i in range(nets):
#     print(max(history[i].history['val_accuracy']))
#     plt.plot(history[i].history['val_accuracy'],linestyle=styles[i])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(names, loc='upper left')
# axes = plt.gca()
# #axes.set_ylim([0.98,1])
# plt.show()


In [ ]:
# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 3
model = [0] *nets

for j in range(3):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,padding='same',activation='relu',
            input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    if j>0:
        model[j].add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D())
    if j>1:
        model[j].add(Conv2D(96,kernel_size=5,padding='same',activation='relu'))
        model[j].add(GlobalMaxPool2D())
    #model[j].add(Flatten())
    if j==0:
        model[j].add(Flatten())
        model[j].add(Reshape((1, 6272)))
    elif j==1:
        model[j].add(Flatten())
        model[j].add(Reshape((1, 3136)))
    else:
        model[j].add(Reshape((1, 96)))

    model[j].add(GRU(128, activation='relu'))

    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])


In [ ]:
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
# TRAIN NETWORKS
history = [0] * nets
names = ["(C-P)x1","(C-P)x2","(C-P)x3"]
epochs = 20
# for j in range(nets):
#     history[j] = model[j].fit(X_train2,Y_train2, batch_size=32, epochs = epochs,
#         validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
#     print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Train loss={2:.5f}, Validation accuracy={3:.5f}, Validation loss={3:.5f}".format(
#         names[j],epochs,max(history[j].history['accuracy']),max(history[j].history['loss']),
#         max(history[j].history['val_accuracy']), max(history[j].history['val_loss']) ))


# # PLOT ACCURACIES
# plt.figure(figsize=(15,5))
# for i in range(nets):
#     #print(max(history[i].history['val_accuracy']))
#     plt.plot(history[i].history['val_accuracy'],linestyle=styles[i])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(names, loc='upper left')
# axes = plt.gca()
# #axes.set_ylim([0.98,1])
# plt.show()
# for i in range(nets):
#     #print(max(history[i].history['val_loss']))
#     plt.plot(history[i].history['val_loss'],linestyle=styles[i])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(names, loc='upper left')
# axes = plt.gca()
# #axes.set_ylim([0.98,1])
# plt.show()

run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)


In [ ]:
# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 6
model = [0] *nets
for j in range(6):
    model[j] = Sequential()
    model[j].add(Conv2D(j*8+8,kernel_size=5,activation='relu',input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    model[j].add(Conv2D(j*16+16,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Flatten())
    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])


In [ ]:
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
# TRAIN NETWORKS
history = [0] * nets
names = ["8 maps","16 maps","24 maps","32 maps","48 maps","64 maps","128 maps"]
# epochs = 20
# for j in range(nets):
#     history[j] = model[j].fit(X_train2,Y_train2, batch_size=32, epochs = epochs,
#         validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
#     print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         names[j],epochs,max(history[j].history['accuracy']),max(history[j].history['val_accuracy']) ))

# # PLOT ACCURACIES
# plt.figure(figsize=(15,5))
# for i in range(nets):
#     print(max(history[i].history['val_accuracy']))
#     plt.plot(history[i].history['val_accuracy'],linestyle=styles[i])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(names, loc='upper left')
# axes = plt.gca()
# #axes.set_ylim([0.98,1])
# plt.show()
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)


In [ ]:
# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 6
model = [0] *nets
for j in range(6):
    model[j] = Sequential()
    model[j].add(Conv2D(j*8+8,kernel_size=5,activation='relu',input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    model[j].add(Conv2D(j*16+16,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Flatten())
    if j == 0:
        model[j].add(Reshape((1,256)))
    elif j == 1:
        model[j].add(Reshape((1,512)))
    elif j == 2:
        model[j].add(Reshape((1,768)))
    elif j == 3:
        model[j].add(Reshape((1,1024)))
    elif j == 4:
        model[j].add(Reshape((1,1280)))
    elif j == 5:
        model[j].add(Reshape((1,1536)))

    model[j].add(GRU(256, activation='relu'))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])


In [ ]:
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
# TRAIN NETWORKS
history = [0] * nets
names = ["8 maps","16 maps","24 maps","32 maps","48 maps","64 maps","128 maps"]
# epochs = 20
# for j in range(nets):
#     history[j] = model[j].fit(X_train2,Y_train2, batch_size=32, epochs = epochs,
#         validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
#     print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         names[j],epochs,max(history[j].history['accuracy']),max(history[j].history['val_accuracy']) ))

# # PLOT ACCURACIES
# plt.figure(figsize=(15,5))
# for i in range(nets):
#     print(max(history[i].history['val_accuracy']))
#     plt.plot(history[i].history['val_accuracy'],linestyle=styles[i])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(names, loc='upper left')
# axes = plt.gca()
# #axes.set_ylim([0.98,1])
# plt.show()
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)


In [ ]:
nets = 8
model = [0] *nets

for j in range(8):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Flatten())
    model[j].add(Dense(128, activation='relu'))
    model[j].add(Dropout(j*0.1))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])

In [ ]:
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
# TRAIN NETWORKS
history = [0] * nets
names = ["0N","32N","64N","128N","256N","512N","1024N","2048N"]
# epochs = 20
# for j in range(nets):
#     history[j] = model[j].fit(X_train2,Y_train2, batch_size=80, epochs = epochs,
#         validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
#     print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         names[j],epochs,max(history[j].history['accuracy']),max(history[j].history['val_accuracy']) ))

# # PLOT ACCURACIES
# plt.figure(figsize=(15,5))
# for i in range(nets):
#     print(max(history[i].history['val_accuracy']))
#     plt.plot(history[i].history['val_accuracy'],linestyle=styles[i])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(names, loc='upper left')
# axes = plt.gca()
# #axes.set_ylim([0.98,1])
# plt.show()
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)


In [ ]:
# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 8
model = [0] *nets

for j in range(8):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
    model[j].add(GlobalMaxPool2D())

    if j>0:
      model[j].add(Reshape((1,64)))
      model[j].add(GRU(2**(j+4), activation='relu'))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])

In [ ]:
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
# TRAIN NETWORKS
history = [0] * nets
names = ["0N","32N","64N","128N","256N","512N","1024N","2048N"]
# epochs = 20
# for j in range(nets):
#     history[j] = model[j].fit(X_train2,Y_train2, batch_size=80, epochs = epochs,
#         validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
#     print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         names[j],epochs,max(history[j].history['accuracy']),max(history[j].history['val_accuracy']) ))

# # PLOT ACCURACIES
# plt.figure(figsize=(15,5))
# for i in range(nets):
#     print(max(history[i].history['val_accuracy']))
#     plt.plot(history[i].history['val_accuracy'],linestyle=styles[i])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(names, loc='upper left')
# axes = plt.gca()
# #axes.set_ylim([0.98,1])
# plt.show()
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2, model)


In [ ]:
nets = 8
model = [0] *nets

for j in range(8):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Flatten())
    model[j].add(Dense(128, activation='relu'))
    model[j].add(Dropout(j*0.1))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])

In [ ]:
 #CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
# TRAIN NETWORKS
history = [0] * nets
names = ["D=0","D=0.1","D=0.2","D=0.3","D=0.4","D=0.5","D=0.6","D=0.7"]
epochs = 30
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)


In [ ]:
# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 8
model = [0] *nets

for j in range(8):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
    model[j].add(GlobalMaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Reshape((1,64)))
    model[j].add(GRU(128, activation='relu'))
    model[j].add(Dropout(j*0.1))
    model[j].add(Dense(9, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    check_com(model[j])


In [ ]:
#CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.333)
# TRAIN NETWORKS
history = [0] * nets
names = ["D=0","D=0.1","D=0.2","D=0.3","D=0.4","D=0.5","D=0.6","D=0.7"]
epochs = 30
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)


In [ ]:
nets = 5
model = [0] *nets

j=0
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Flatten())
model[j].add(Dense(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=1
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Flatten())
model[j].add(Dense(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=2
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Flatten())
model[j].add(Dense(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=3
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Flatten())
model[j].add(Dense(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=4
model[j] = Sequential()

model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))

model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))

model[j].add(Flatten())
model[j].add(Dense(128, activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))

model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])


In [ ]:
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.2)
# TRAIN NETWORKS 1,2,3,4
history = [0] * nets
names = ["basic","32C3-32C3","32C5S2","both+BN","both+BN+DA"]
epochs = 100
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)


In [ ]:
nets = 5
model = [0] *nets

j=0
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Reshape((1,1024)))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=1
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Reshape((1,1024)))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=2
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Flatten())
model[j].add(Reshape((1,1024)))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=3
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Flatten())
model[j].add(Reshape((1,1024)))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=4
model[j] = Sequential()

model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))

model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Flatten())
model[j].add(Reshape((1,1024)))
model[j].add(GRU(128, activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))

model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])


In [ ]:
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.2)
# TRAIN NETWORKS 1,2,3,4
history = [0] * nets
names = ["basic","32C3-32C3","32C5S2","both+BN","both+BN+DA"]
epochs = 100
run_and_plot(names, annealer, nets, styles, X_train2, X_val2, Y_train2, Y_val2 , model)

In [ ]:
nets = 5
model = [0] *nets

j=0
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
model[j].add(GlobalMaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Reshape((1,64)))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=1
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(MaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(globalMaxPool2D())
model[j].add(Dropout(0.4))
model[j].add(Reshape((1,64)))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=2
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(globalMaxPool2D())
model[j].add(Reshape((1,64)))
model[j].add(Dropout(0.4))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=3
model[j] = Sequential()
model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(globalMaxPool2D())
model[j].add(Reshape((1,64)))
model[j].add(Dropout(0.4))
model[j].add(GRU(128, activation='relu'))
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])

j=4
model[j] = Sequential()

model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))

model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))

model[j].add(globalMaxPool2D())
model[j].add(Reshape((1,64)))
model[j].add(Dropout(0.4))
model[j].add(GRU(128, activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Dense(9, activation='softmax'))

model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
check_com(model[j])


In [ ]:
# prompt: write a code to go through this kaggle directory " kaggle /kaggle/working/ " of this website "https://www.kaggle.com/code/maminu0/notebook4b85fc2fb0/edit" and dowload all available files to my desktop

import os
import requests
from bs4 import BeautifulSoup

def download_kaggle_files(url, directory):
  # Send an HTTP GET request to the URL
  response = requests.get(url)

  # Parse the HTML content of the page
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find all links to files within the specified directory
  file_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith(directory)]

  # Download each file
  for file_link in file_links:
    file_url = 'https://www.kaggle.com' + file_link
    file_name = os.path.basename(file_link)
    file_path = os.path.join('/content', file_name)  # Assuming you want to download to Colab's /content directory

    print(f"Downloading {file_name}...")
    response = requests.get(file_url)

    with open(file_path, 'wb') as f:
      f.write(response.content)

    print(f"Downloaded {file_name} to {file_path}")

# Example usage
kaggle_url = 'https://www.kaggle.com/code/maminu0/notebook4b85fc2fb0/edit'
kaggle_directory = 'kaggle/working/'

download_kaggle_files(kaggle_url, kaggle_directory)

# After downloading to Colab, you can use the following to download to your local machine
from google.colab import files
for file_name in os.listdir('/content'):
  if file_name.startswith('kaggle_working_'):
    files.download(os.path.join('/content', file_name))


In [ ]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
      if the_file.endswith(".png"):
          file_path = os.path.join(folder, the_file)
          try:
              if os.path.isfile(file_path):
                  os.unlink(file_path)
              elif os.path.isdir(file_path):
                  remove_folder_contents(file_path)
                  os.rmdir(file_path)
          except Exception as e:
              print(e)
      else:
        continue

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPool2D, GlobalMaxPool2D, Reshape, GRU, Dense, Attention

j = 0
model[j] = Sequential()

model[j].add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(64, 64, 1)))
model[j].add(Dropout(0.1))
model[j].add(MaxPool2D(2))

model[j].add(Conv2D(64, kernel_size=3, activation='relu'))
model[j].add(Dropout(0.1))
model[j].add(MaxPool2D(2))

model[j].add(Conv2D(96, kernel_size=3, activation='relu'))
model[j].add(Conv2D(96, kernel_size=3, activation='relu'))
model[j].add(Dropout(0.3))

model[j].add(GlobalMaxPool2D())
model[j].add(Reshape((1, 96)))

# Replace the Bidirectional GRU with a GRU followed by an Attention layer
model[j].add(GRU(256, activation='relu', return_sequences=True))

# Apply self-attention by passing the same tensor as query and value
def apply_attention(inputs):
    # inputs is a tensor with shape (batch_size, timesteps, features)
    # We apply self-attention, so query and value are the same
    query = value = inputs
    key = None  # For self-attention, key can be omitted
    attention_output = Attention()([query, value, key])
    return attention_output

# Use a Lambda layer to apply the attention function
from tensorflow.keras.layers import Lambda
model[j].add(Lambda(apply_attention))

# model[j].add(BatchNormalization())
model[j].add(Dropout(0.1))
model[j].add(Dense(9, activation='softmax'))

model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# prompt: Modify this code to include attention mechanism
# j=6
# model[j] = Sequential()
# model[j].add(Conv2D(64,kernel_size=3,activation='relu',input_shape=(64,64,1)))
# model[j].add(Dropout(0.1))
# # model[j].add(BatchNormalization())
# model[j].add(MaxPool2D(2))
# model[j].add(Conv2D(96,kernel_size=3,activation='relu'))
# model[j].add(Dropout(0.1))
# # model[j].add(BatchNormalization())
# model[j].add(MaxPool2D(2))

j=0
model[j] = Sequential()
model[j].add(Conv2D(64,kernel_size=3,activation='relu',input_shape=(64,64,1)))
model[j].add(Dropout(0.1))
# model[j].add(BatchNormalization())
model[j].add(MaxPool2D(2))
model[j].add(Conv2D(96,kernel_size=3,activation='relu'))
model[j].add(Dropout(0.1))
# model[j].add(BatchNormalization())
model[j].add(MaxPool2D(2))
model[j].add(Conv2D(128, kernel_size=3, activation='relu'))
model[j].add(Conv2D(128, kernel_size=3, activation='relu'))
model[j].add(Dropout(0.3))
model[j].add(GlobalMaxPool2D())
model[j].add(Reshape((1, 128)))
model[j].add(GRU(256, activation='relu', return_sequences=True))
# Apply self-attention by passing the same tensor as query and value
def apply_attention(inputs):
    # inputs is a tensor with shape (batch_size, timesteps, features)
    # We apply self-attention, so query and value are the same
    query = value = inputs = key
    key = None  # For self-attention, key can be omitted
    attention_output = Attention()([query, value, key])
    return attention_output
# Use a Lambda layer to apply the attention function
model[j].add(Lambda(apply_attention))
# model[j].add(BatchNormalization())
model[j].add(Dropout(0.1))
model[j].add(Dense(9, activation='softmax'))
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
model[j].summary()

In [ ]:
def plot_(hist, name="finals_pattern_plot"):
    fig , ax = plt.subplots(1,2)
    train_acc = hist.history['accuracy']
    train_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    fig.set_size_inches(16,9)

    ax[0].plot( train_acc , 'go-' , label = 'Training Accuracy')
    ax[0].plot( val_acc , 'ro-' , label = 'Testing Accuracy')
    ax[0].set_title('Training & Validation Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")

    ax[1].plot( train_loss , 'g-o' , label = 'Training Loss')
    ax[1].plot( val_loss , 'r-o' , label = 'Testing Loss')
    ax[1].set_title('Testing Accuracy & Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    plt.savefig(f"{'_'.join(name)}_{str(time.time())}_{name}_.png")

def pre_cl(model,X_val2,Y_val2, name="finals"):
    predictions = model.predict(X_val2)
    Xpred = np.argmax(predictions, axis=1)
    ytrue = np.argmax(Y_val2, axis=1)
    classes = ["Class " + str(i) for i in range(9) if i != 9]
    print(classification_report(ytrue, Xpred))#, target_names = classes)
    correct = np.nonzero(Xpred == ytrue)[0]
    plt.figure(figsize = (15,15))
    i = 0
    for c in correct[:6]:
        plt.subplot(3,2,i+1)
        plt.imshow(X_val2[c], cmap="gray", interpolation='none')
        plt.title("Predicted Class {},Actual Class {}".format(Xpred[c], ytrue[c]))
        plt.tight_layout()
        i += 1
    plt.savefig(f"{'_'.join(name)}_{str(time.time())}_{name}_.png")
    cm = confusion_matrix(ytrue,Xpred)
    sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')



In [ ]:
plot_(hist6, name=names[6])
pre_cl(model[6],X_val2,Y_val2, name=names[6])

In [ ]:
Modify the code below to use multi-head attention from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.layers import MultiHeadAttention, Input, Dense, Flatten, Conv2D, Dropout, MaxPool2D, GlobalMaxPool2D, Reshape, GRU, LSTM
from tensorflow.keras.models import Model

j=0
# Use the Functional API to create a model with multiple inputs
input_tensor = Input(shape=(28, 28, 1))
x = Conv2D(32,kernel_size=3,activation='relu')(input_tensor)
x = Dropout(0.1)(x)
x = MaxPool2D(2)(x)
x = Conv2D(64,kernel_size=3,activation='relu')(x)
x = Dropout(0.1)(x)
x = MaxPool2D(2)(x)
x = Conv2D(96, kernel_size=3, activation='relu')(x)
x = Conv2D(96, kernel_size=3, activation='relu')(x)
x = Dropout(0.3)(x)
x = GlobalMaxPool2D()(x)
x = Reshape((1, 96))(x)
x = LSTM(256, activation='relu', return_sequences=True)(x)

# Use the same tensor as both 'query' and 'value' for self-attention
attention_output = MultiHeadAttention(num_heads=8, key_dim=128)(x, x)

x = Dropout(0.1)(attention_output)
x = Flatten()(x)
output_tensor = Dense(9, activation='softmax')(x)

model[j] = Model(inputs=input_tensor, outputs=output_tensor)
model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model[j].summary()


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.3, stratify=Y_train, random_state=42)
#hist6 = model[0].fit(X_train2,Y_train2, epochs = 200, batch_size=128, verbose=0, validation_data=(X_val2,Y_val2))

In [ ]:
X_train2.shape

In [ ]:
import time

hist6 = model[0].fit(X_train2,Y_train2, epochs = 200, batch_size=128, verbose=0, validation_data=(X_val2,Y_val2))

In [ ]:
import seaborn as sns
plot_(hist6)#, name=names[0])
pre_cl(model[0],X_val2,Y_val2)#, name=names[0])

In [ ]:
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train_, Y_train_, test_size = 0.333)


In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dropout, MaxPool2D, GlobalMaxPool2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

# Hyperparameters search space
units = [8, 16, 32, 64]#, 96, 128]
kernel_sizes = [2,3]#, 5]#, 7]
activation = ['relu', 'elu', 'selu']
dropout = [0.1, 0.2, 0.3, 0.4, 0.5]
maxpool = [2, 3]

# Example Dataset (For simplicity, using random data)
#X = np.random.rand(1000, 28, 28, 1)
#y = to_categorical(np.random.randint(9, size=1000))

# Split data into training and validation sets
X_train, X_val, y_train, y_val = X_train2, X_val2, Y_train2, Y_val2#train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Define the ABC Algorithm
class ABCAlgorithm:
    def __init__(self, num_bees, max_iterations, dataset, labels, limit=10):
        self.num_bees = num_bees
        self.max_iterations = max_iterations
        self.dataset = dataset
        self.labels = labels
        self.limit = limit
        # Initialize random solutions (hyperparameter combinations)
        self.population = [self.random_solution() for _ in range(self.num_bees)]
        self.fitness = np.zeros(self.num_bees)
        self.trial_counter = np.zeros(self.num_bees)

    def random_solution(self):
        return {
            'units': np.random.choice(units),
            'kernel_size': np.random.choice(kernel_sizes),
            'activation': np.random.choice(activation),
            'dropout': np.random.choice(dropout),
            'maxpool': np.random.choice(maxpool)
        }

    def evaluate_fitness(self):
        for i in range(self.num_bees):
            self.fitness[i] = self.evaluate_single_fitness(self.population[i])

    def employed_bees_phase(self):
        for i in range(self.num_bees):
            new_solution = self.mutate_solution(self.population[i])
            new_fitness = self.evaluate_single_fitness(new_solution)
            if new_fitness > self.fitness[i]:
                self.population[i] = new_solution
                self.fitness[i] = new_fitness
                self.trial_counter[i] = 0
            else:
                self.trial_counter[i] += 1

    def onlooker_bees_phase(self):
        fitness_prob = self.fitness / self.fitness.sum()
        for i in range(self.num_bees):
            selected_bee = np.random.choice(range(self.num_bees), p=fitness_prob)
            new_solution = self.mutate_solution(self.population[selected_bee])
            new_fitness = self.evaluate_single_fitness(new_solution)
            if new_fitness > self.fitness[selected_bee]:
                self.population[selected_bee] = new_solution
                self.fitness[selected_bee] = new_fitness
                self.trial_counter[selected_bee] = 0
            else:
                self.trial_counter[selected_bee] += 1

    def scout_bees_phase(self):
        for i in range(self.num_bees):
            if self.trial_counter[i] > self.limit:
                self.population[i] = self.random_solution()
                self.fitness[i] = self.evaluate_single_fitness(self.population[i])
                self.trial_counter[i] = 0

    def mutate_solution(self, solution):
        new_solution = solution.copy()
        param_to_mutate = np.random.choice(list(solution.keys()))
        if param_to_mutate == 'units':
            new_solution['units'] = np.random.choice(units)
        elif param_to_mutate == 'kernel_size':
            new_solution['kernel_size'] = np.random.choice(kernel_sizes)
        elif param_to_mutate == 'activation':
            new_solution['activation'] = np.random.choice(activation)
        elif param_to_mutate == 'dropout':
            new_solution['dropout'] = np.random.choice(dropout)
        elif param_to_mutate == 'maxpool':
            new_solution['maxpool'] = np.random.choice(maxpool)
        return new_solution

    def build_and_compile_model(self, hyperparams):

        kernel_size_tuple = (hyperparams['kernel_size'], hyperparams['kernel_size'])  # Convert kernel_size to a tuple
        pool_size_tuple = (hyperparams['maxpool'], hyperparams['maxpool'])  # Convert kernel_size to a tuple
        input_tensor = Input(shape=(64, 64, 1))
        x = Conv2D(hyperparams['units'], kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(input_tensor)
        x = Dropout(hyperparams['dropout'])(x)
        x = MaxPool2D(pool_size_tuple)(x)
        x = Conv2D(hyperparams['units']+32, kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(x)
        x = Dropout(hyperparams['dropout'])(x)
        x = MaxPool2D(pool_size_tuple)(x)
        x = Conv2D(hyperparams['units']+64, kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(x)
        x = Conv2D(hyperparams['units']+64, kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(x)
        x = Dropout(hyperparams['dropout'])(x)
        x = GlobalMaxPool2D()(x)
        x = Reshape((1, hyperparams['units']+64))(x)
        x = Bidirectional(LSTM(hyperparams['units']+64))(x)
        x = Dropout(hyperparams['dropout'])(x)
        x = Flatten()(x)
        output_tensor = Dense(9, activation='softmax')(x)

        model = Model(inputs=input_tensor, outputs=output_tensor)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        return model


    def evaluate_single_fitness(self, hyperparams):
        model = self.build_and_compile_model(hyperparams)
        model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=0)  # Use smaller epochs for faster evaluation
        loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
        return accuracy

    def optimize(self):
        for iteration in range(self.max_iterations):
            self.evaluate_fitness()
            self.employed_bees_phase()
            self.onlooker_bees_phase()
            self.scout_bees_phase()
            best_bee_idx = np.argmax(self.fitness)
            best_solution = self.population[best_bee_idx]
            print(f"Iteration {iteration}: Best Fitness = {self.fitness[best_bee_idx]}")
        return best_solution

# Define the GTO Algorithm
class GTOAlgorithm:
    def __init__(self, solution, dataset, labels):
        self.solution = solution
        self.dataset = dataset
        self.labels = labels

    def refine(self):
        selected_solution = self.solution
        model = self.build_and_compile_model(selected_solution)

        model.fit(X_train, y_train, epochs=30, batch_size=128, verbose=0)  # Fine-tuning with more epochs
        loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
        print(f"Refined Solution Accuracy: {accuracy}")
        return selected_solution


    def build_and_compile_model(self, hyperparams):
      pool_size_tuple = (hyperparams['maxpool'], hyperparams['maxpool'])  # Convert kernel_size to a tuple
      kernel_size_tuple = (hyperparams['kernel_size'], hyperparams['kernel_size'])  # Convert kernel_size to a tuple
      input_tensor = Input(shape=(64, 64, 1))
      x = Conv2D(hyperparams['units'], kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(input_tensor)
      x = Dropout(hyperparams['dropout'])(x)
      x = MaxPool2D(pool_size_tuple)(x)
      x = Conv2D(hyperparams['units']+32, kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(x)
      x = Dropout(hyperparams['dropout'])(x)
      x = MaxPool2D(pool_size_tuple)(x)
      x = Conv2D(hyperparams['units']+64, kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(x)
      x = Conv2D(hyperparams['units']+64, kernel_size=kernel_size_tuple, activation=hyperparams['activation'])(x)
      x = Dropout(hyperparams['dropout'])(x)
      x = GlobalMaxPool2D()(x)
      x = Reshape((1, hyperparams['units']+64))(x)
      x = Bidirectional(LSTM(hyperparams['units']+64))(x)
      x = Dropout(hyperparams['dropout'])(x)
      x = Flatten()(x)
      output_tensor = Dense(9, activation='softmax')(x)

      model = Model(inputs=input_tensor, outputs=output_tensor)
      model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
      return model


# Combine ABC and GTO
def hybrid_abc_gto(dataset, labels, num_bees=10, max_iterations=5):
    abc = ABCAlgorithm(num_bees, max_iterations, dataset, labels)
    best_solution = abc.optimize()

    gto = GTOAlgorithm(best_solution, dataset, labels)
    refined_solution = gto.refine()

    return refined_solution

# Example usage


In [ ]:
# Hyperparameters search space
units = [16, 32, 64]#, 96, 128]
kernel_sizes = [2,3]#, 5]#, 7]
activation = ['relu', 'elu', 'selu']
dropout = [0.1, 0.2, 0.3, 0.4, 0.5]
maxpool = [2, 3]
best_hyperparameters = hybrid_abc_gto(X_train_, Y_train_)
print("Best Hyperparameters found:", best_hyperparameters)


In [ ]:
# Hyperparameters search space
units = [8, 16, 32, 64]#, 96, 128]
kernel_sizes = [2,3]#, 5]#, 7]
activation = ['relu', 'elu', 'selu']
dropout = [0.1, 0.2, 0.3, 0.4, 0.5]
maxpool = [2, 3]

best_hyperparameters = hybrid_abc_gto(X_train_, Y_train_)
print("Best Hyperparameters found:", best_hyperparameters)

In [ ]:
Best Hyperparameters found: {'units': 32, 'kernel_size': 3, 'activation': 'relu', 'dropout': 0.1, 'maxpool': 2}
Best Hyperparameters found: {'units': 32, 'kernel_size': 3, 'activation': 'relu', 'dropout': 0.2, 'maxpool': 2}

